In [ ]:
# Downloads a checkpoint file.
#
# It is advisable to use this file from Google Drive because the server used
# within the `piano_transcription_inference` Python package for downloading has
# slow download speeds (approximately 2mbps).
#
!gdown '1F3jUW2WQyIGbiPqUTcyE4zpd38v0sY7V'

In [ ]:
!pip install --upgrade \
  git+https://github.com/librosa/librosa.git@0.8.1 \
  git+https://github.com/qiuqiangkong/piano_transcription_inference.git \
  git+https://github.com/ytdl-org/youtube-dl.git \
  git+https://github.com/spotDL/spotify-downloader.git \
  > /dev/null 2>&1

In [ ]:
!rm -rf *.mp3

# ---
# You can use 'youtube-dl' to download audio/video files from YouTube!
# ---

# !youtube-dl -x --audio-format 'mp3' -f '(bestvideo+bestaudio/best)[protocol^=http][ext=mp4]' 'https://www.youtube.com/playlist?list=OLAK5uy_nDowl3ctSx-IyCvpXPaBeSLzVCjjKQUrA'

# ---
# Or from Spotify too!
# Uncomment those lines below.
# ---

!spotdl download 'https://open.spotify.com/playlist/0XUG1CSzpasBfhwj8PcaWk' --format mp3 > /dev/null 2>&1

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, HTML
from piano_transcription_inference import PianoTranscription, sample_rate, load_audio
import datetime
import base64
import mimetypes


# Function to transcribe audio and save as MIDI
def transcribe_to_midi(audio_file_path):
    """
    Transcribe the input audio file to MIDI format and save the MIDI file.

    Args:
        audio_file_path (str): Path to the input audio file.

    Returns:
        str: Path to the generated MIDI file.
    """

    # Get the current UTC date and time as a formatted string
    current_utc_time = datetime.datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')

    # Create a MIDI file name with the current UTC date and time
    midi_file_path = f'{audio_file_path}_{current_utc_time}.mid'

    (audio, _) = load_audio(audio_file_path, sr=sample_rate, mono=True)

    transcriptor = PianoTranscription(
        device='cuda',
        checkpoint_path='/content/note_F1=0.9677_pedal_F1=0.9186.pth')
    transcribed_dict = transcriptor.transcribe(audio, midi_file_path)

    return midi_file_path


# Function to get a list of MP3 files in the current directory (case-insensitive)
def get_mp3_files_in_current_directory():
    """
    Get a list of MP3 files in the current directory (case-insensitive).

    Returns:
        list: List of MP3 file names.
    """
    mp3_files = []
    for filename in os.listdir('.'):
        if filename.lower().endswith('.mp3'):
            mp3_files.append(filename)
    return mp3_files


# Iterate through MP3 files and transcribe them to MIDI
mp3_files = get_mp3_files_in_current_directory()
for mp3_file in mp3_files:
    print(f"Processing MP3 file: {mp3_file}")
    midi_file_path = transcribe_to_midi(mp3_file)

    def get_mime_type(file_path):
        """
        Get the MIME type of a file based on its extension.

        Args:
            file_path (str): Path to the file.

        Returns:
            str: MIME type.
        """
        mime, _ = mimetypes.guess_type(file_path)
        return mime or 'application/octet-stream'

    def file_to_base64_uri(file_path):
        """
        Convert a file to a base64-encoded data URI.

        Args:
            file_path (str): Path to the file.

        Returns:
            str: Base64-encoded data URI.
        """
        try:
            with open(file_path, 'rb') as file:
                file_content = file.read()
                base64_data = base64.b64encode(file_content).decode('utf-8')
                base64_uri = f"data:{get_mime_type(file_path)};base64,{base64_data}"
                return base64_uri
        except FileNotFoundError:
            return "File not found"
        except Exception as e:
            return str(e)

    base64_uri = file_to_base64_uri(midi_file_path)

    display(
        HTML(
            f'<a href="{base64_uri}" download="{midi_file_path}"><button>Download MIDI File</button></a>'
        ))
